In [2]:
import pandas as pd
import configparser
import pandas as pd
import numpy as np
from sqlmodel import SQLModel, Field, Relationship, create_engine

engine = create_engine("sqlite:///storage/invoice_tracker_null.db")

import re
def normalize(text):
    if pd.isna(text):
        return ""
    text = text.strip().lower()
    text = re.sub(r'[^a-z0-9]', '', text)  # remove all non-alphanumeric characters
    return text




In [24]:
import pandas as pd
import oracledb
from config import oracle_config
oracledb.init_oracle_client(lib_dir=r"C:\oracle\instantclient_23_8")
from datetime import datetime



class DataFetcherOracleSupplier:
    
    def __init__(self,psswrd,user_name,dsn):
        self.connection = None
        self.cursor = None
        self.psswrd=psswrd
        self.user_name=user_name
        self.dsn=dsn

    def connect(self):
        try:
            self.connection = oracledb.connect(
                user=self.user_name,
                password=self.psswrd,
                dsn=self.dsn
            )
            self.cursor = self.connection.cursor()
            print("connection established")
        except Exception as e:
            raise ConnectionError(f"❌ Failed to connect to Oracle DB: {e}")    



    def disconnect(self):
        if self.cursor:
            self.cursor.close()
        if self.connection:
            self.connection.close()
                

   



    def fetch_invoice_data(self,date:str)-> pd.DataFrame:
        try:
            if not self.connection:
                raise ConnectionError("Database connection is not established.")    
            
            self.cursor.execute("""
                    BEGIN
                        mo_global.set_policy_context('S','82');
                    END;
                    """)
            self.connection.commit() 
            today = datetime.now().strftime('%d-%b-%Y').upper()

            query = f"""
                        SELECT
                            AP.INVOICE_DATE "Invoice Date",
                            AP.GL_DATE "Trx Date",
                            apv.VENDOR_TYPE_DISP, AP.VENDOR_ID, AP.VENDOR_SITE_ID, AP.VENDOR_NUMBER,
                            AP.VENDOR_NAME "Supplier Name",
                            APV.VAT_registration_num "VAT No",
                            AP.INVOICE_NUM " Invoice Number",
                            AP.SUPPLIER_TAX_INVOICE_NUMBER  "Control Unit Invoice Number",
                            (SELECT USER_NAME FROM FND_USER WHERE USER_ID = AP.CREATED_BY) "Created By",
                            AP.DESCRIPTION " Item Description",
                            AP.INVOICE_CURRENCY_CODE "Invoice Currency",
                            AP.EXCHANGE_RATE "Exchange Rate",
                            AP.CREDITED_INVOICE_NUM,
                            AP.PO_NUMBER,
                            AP.RECEIPT_NUMBER,
                            AP.DISTRIBUTION_TOTAL,
                            CASE WHEN AP.DISTRIBUTION_TOTAL = 0 THEN 'N' ELSE 'Y' END "Distribution Flag",
                            CASE WHEN AP.INVOICE_ID IN (
                                SELECT DISTINCT PK1_VALUE FROM FND_ATTACHED_DOCUMENTS WHERE ENTITY_NAME = 'AP_INVOICES'
                            ) THEN 'Yes' ELSE 'No' END "Attachments",
                            CASE WHEN (AP.TOTAL_TAX_AMOUNT <> 0)
                                THEN (AP.ACTUAL_INVOICE_AMOUNT - AP.TOTAL_TAX_AMOUNT)
                                ELSE AP.ACTUAL_INVOICE_AMOUNT END "Exclusive",
                            AP.TOTAL_TAX_AMOUNT "Tax",
                            AP.ACTUAL_INVOICE_AMOUNT "Inclusive",
                            CASE WHEN (AP.INVOICE_CURRENCY_CODE = 'USD')
                                THEN ((AP.ACTUAL_INVOICE_AMOUNT - AP.TOTAL_TAX_AMOUNT) * AP.EXCHANGE_RATE)
                                ELSE (AP.ACTUAL_INVOICE_AMOUNT - AP.TOTAL_TAX_AMOUNT) END "Exclusive KSH",
                            CASE WHEN (AP.INVOICE_CURRENCY_CODE = 'USD')
                                THEN (AP.TOTAL_TAX_AMOUNT * AP.EXCHANGE_RATE)
                                ELSE AP.TOTAL_TAX_AMOUNT END "Tax KSH",
                            CASE WHEN (AP.INVOICE_CURRENCY_CODE = 'USD')
                                THEN (AP.ACTUAL_INVOICE_AMOUNT * AP.EXCHANGE_RATE)
                                ELSE AP.ACTUAL_INVOICE_AMOUNT END "Inclusive KSH"
                        FROM
                            AP_invoices_V AP,
                            AP_vendors_v APV
                        WHERE
                            TO_DATE(AP.GL_DATE,'DD-MON-RRRR') >= TO_DATE('{date}', 'DD-MON-RRRR')
                            AND TO_DATE(AP.GL_DATE,'DD-MON-RRRR') <= TO_DATE('{today}', 'DD-MON-RRRR')
                            AND AP.VENDOR_NUMBER = APV.VENDOR_NUMBER
                        """

            df = self.cursor.execute(query)
            rows = df.fetchall()
            columns = [desc[0] for desc in self.cursor.description]  # Get column n
            data = pd.DataFrame(rows, columns=columns)
            data['invoiceUniqueKey']=data['VENDOR_ID'].astype(str)+"_"+data[' Invoice Number']
            final_data=data[['invoiceUniqueKey','Invoice Date','VENDOR_ID',' Invoice Number','Inclusive KSH']]
            final_data=final_data[final_data['Invoice Date']>date]
            final_data=final_data[final_data['Invoice Date']<'12-02-2024']

            
            
            cleaned=final_data.rename(columns={
                                                ' Invoice Number': 'invoiceNumber',
                                                'Invoice Date': 'invoiceDate',
                                                'VENDOR_ID': 'vendorId',
                                                'Inclusive KSH':'invoiceGrossValue'
                                            })
            return cleaned.drop_duplicates()
            
        

        except Exception as e:
            raise RuntimeError(f"❌ Failed to fetch invoice data: {e}")

        


        


    def fetch_customer_data(self,vendor_ids:list) -> pd.DataFrame :
        vendor_ids=tuple(int(i) for i in vendor_ids)
        try:
            if not self.connection:
                raise ConnectionError("Database connection is not established.")

            if not vendor_ids:
                raise ValueError("Customer number list is empty.")    
        


            query=f"""select
aps.vendor_id   "SUPPLIER ID",aps.END_DATE_ACTIVE,
aps.vendor_name "SUPPLIER NAME",aps.CREATION_DATE,fu.USER_NAME,
aps.segment1  "SUPPLIER NUMBER",
-------
--PER.EMPLOYEE_NUMBER,PER.GLOBAL_NAME,pp.D_TERMINATION_DATE,PER.PERSON_ID,
--------
aps.vendor_type_lookup_code "SUPPLIER TYPE",
apss.VENDOR_SITE_ID,
apss.VENDOR_SITE_CODE "SUPPLIER SITE",
apss.CREATE_DEBIT_MEMO_FLAG,--added by Godfrey 28-06-2019  RTV issue
aps.CREATE_DEBIT_MEMO_FLAG,  --added by Godfrey 28-06-2019  RTV issue
aps.terms_id ,
aps.vat_registration_num  "TAX NUMBER",
DECODE(aps.auto_tax_calc_flag, 'N', 'No', 'Y', 'Yes', aps.auto_tax_calc_flag) "AutoCalculate Tax Flag",
apss.ALLOW_AWT_FLAG,
aps.party_id,
apt.name "PAY TERMS",
PVC.title,
pvc.prefix||' '||pvc.first_name||' '||pvc.middle_name||' '||pvc.last_name "CONTACT NAME",
pvc.mail_stop,
pvc.area_code,
pvc.phone,
pvc.email_address,
pvc.alt_area_code,
pvc.alt_phone,
pvc.fax_area_code,
pvc.fax,
apss.address_line1,
apss.address_line2,
apss.address_line3,
apss.city,
apss.state,
apss.zip,
apss.country,
apss.invoice_currency_code "INV CURR",
apss.payment_currency_code "PAY CURR",
-------------------------
(select bc.BANK_CHARGE_BEARER from IBY_EXTERNAL_PAYEES_ALL bc --AP_SUPPLIER_SITES_ALL apss
where apss.VENDOR_SITE_ID = bc.SUPPLIER_SITE_ID)"BANK_CHARGE_BEARER",
----------------------------
(SELECT
ieb.bank_name
FROM
apps.iby_ext_bank_accounts ieba,
apps.iby_account_owners iao,
apps.iby_ext_banks_v ieb,
apps.iby_ext_bank_branches_v iebb
WHERE 1=1
and aps.vendor_id = apss.vendor_id
and iao.account_owner_party_id = aps.party_id
and ieba.ext_bank_account_id = iao.ext_bank_account_id
and ieb.bank_party_id = iebb.bank_party_id
and ieba.branch_id = iebb.branch_party_id
and ieba.bank_id = ieb.bank_party_id
and ROWNUM=1)"BANK  NAME",

(SELECT
iebb.bank_branch_name
FROM
apps.iby_ext_bank_accounts ieba,
apps.iby_account_owners iao,
apps.iby_ext_banks_v ieb,
apps.iby_ext_bank_branches_v iebb
WHERE 1=1
and aps.vendor_id = apss.vendor_id
and iao.account_owner_party_id = aps.party_id
and ieba.ext_bank_account_id = iao.ext_bank_account_id
and ieb.bank_party_id = iebb.bank_party_id
and ieba.branch_id = iebb.branch_party_id
and ieba.bank_id = ieb.bank_party_id
and ROWNUM=1)"BANK BRANCH NAME",
--------
(SELECT
iebb.BRANCH_NUMBER
FROM
apps.iby_ext_bank_accounts ieba,
apps.iby_account_owners iao,
apps.iby_ext_banks_v ieb,
apps.iby_ext_bank_branches_v iebb
WHERE 1=1
and aps.vendor_id = apss.vendor_id
and iao.account_owner_party_id = aps.party_id
and ieba.ext_bank_account_id = iao.ext_bank_account_id
and ieb.bank_party_id = iebb.bank_party_id
and ieba.branch_id = iebb.branch_party_id
and ieba.bank_id = ieb.bank_party_id
and ROWNUM=1)"BRANCH NUM",
----------
(SELECT
iebb.EFT_SWIFT_CODE
FROM
apps.iby_ext_bank_accounts ieba,
apps.iby_account_owners iao,
apps.iby_ext_banks_v ieb,
apps.iby_ext_bank_branches_v iebb
WHERE 1=1
and aps.vendor_id = apss.vendor_id
and iao.account_owner_party_id = aps.party_id
and ieba.ext_bank_account_id = iao.ext_bank_account_id
and ieb.bank_party_id = iebb.bank_party_id
and ieba.branch_id = iebb.branch_party_id
and ieba.bank_id = ieb.bank_party_id
and ROWNUM=1)"EFT_BIC",
(SELECT
ieba.BANK_ACCOUNT_NUM
--ieba.BANK_ACCOUNT_NAME "BANK ACCOUNT NAME"
FROM
apps.iby_ext_bank_accounts ieba,
apps.iby_account_owners iao,
apps.iby_ext_banks_v ieb,
apps.iby_ext_bank_branches_v iebb
WHERE 1=1
and aps.vendor_id = apss.vendor_id
and iao.account_owner_party_id = aps.party_id
and ieba.ext_bank_account_id = iao.ext_bank_account_id
and ROWNUM=1)"BANK ACCOUNT NUMBER",
(SELECT
ieba.BANK_ACCOUNT_NAME "BANK ACCOUNT NAME"
FROM
apps.iby_ext_bank_accounts ieba,
apps.iby_account_owners iao,
apps.iby_ext_banks_v ieb,
apps.iby_ext_bank_branches_v iebb
WHERE 1=1
and aps.vendor_id = apss.vendor_id
and iao.account_owner_party_id = aps.party_id
and ieba.ext_bank_account_id = iao.ext_bank_account_id
and ROWNUM=1)"BANK ACCOUNT NAME",
aps.MATCH_OPTION,receipt_required_flag,aps.INSPECTION_REQUIRED_FLAG,
DECODE (NVL(APS.receipt_required_flag, 'N'),
'Y', DECODE(NVL(APS.inspection_required_flag, 'N'),
'Y', '4-Way',
'3-Way'),
'2-Way') matching_Level
from ap_suppliers aps,
ap_supplier_sites_all apss,FND_USER FU,
po_vendor_contacts pvc,ap_terms apt
-------------------

-------------------------
where aps.vendor_id=apss.vendor_id(+)
and apss.vendor_id=pvc.vendor_id(+)
and aps.TERMS_ID=apt.TERM_ID(+)
AND aps.vendor_type_lookup_code <> 'EMPLOYEE'
-------------------------
--AND APS.EMPLOYEE_ID = PER.PERSON_ID
--AND APS.EMPLOYEE_ID = Pp.PERSON_ID
-----------------------------
--AND APS.EMPLOYEE_ID ='688'
--AND aps.VENDOR_NAME = NVL (:P_VENDOR_NAME, aps.VENDOR_NAME)
--AND pp.D_TERMINATION_DATE IS NULL
--and apss.ALLOW_AWT_FLAG is null
--and aps.END_DATE_ACTIVE is null
--and apss.CREATE_DEBIT_MEMO_FLAG = 'N'
--and aps.CREATE_DEBIT_MEMO_FLAG = 'N'
--AND aps.CREATION_DATE BETWEEN NVL (:p_from_date, aps.CREATION_DATE)
--AND NVL (:p_to_date, aps.CREATION_DATE)+1 
and aps.LAST_UPDATED_BY = fu.USER_ID
and aps.vendor_id in {vendor_ids}
--and apt.name is null --Pay Terms

order by aps.segment1 """
            df = self.cursor.execute(query)
            rows = df.fetchall()
            print("Sucessfully fetched data")
            columns = [desc[0] for desc in self.cursor.description]  # Get column names

            
            data = pd.DataFrame(rows, columns=columns)
            
        
            
            data=data[['SUPPLIER ID','SUPPLIER NAME','SUPPLIER TYPE','PAY TERMS', 'PHONE', 'EMAIL_ADDRESS']]
            print(data.columns)
            cleaned = data.rename(columns={
                                            'SUPPLIER ID': 'vendorId',
                                            'SUPPLIER NAME': 'supplierName',
                                            'EMAIL_ADDRESS': 'email',
                                            'PHONE': 'phone',
                                            'SUPPLIER TYPE': 'supplierType',
                                            'PAY TERMS': 'creditTerms'
                                        })
            
            print(cleaned.columns)
            return cleaned.drop_duplicates()
            

        except Exception as e:
            raise RuntimeError(f"❌ Failed to fetch customer data: {e}")

In [25]:
dsn, username, password=oracle_config()

fetch=DataFetcherOracleSupplier(password,username,dsn)
fetch.connect()

connection established


In [16]:
data=fetch.fetch_invoice_data('20-JUN-2025')

In [17]:
data

,invoiceUniqueKey,invoiceDate,vendorId,invoiceNumber,invoiceGrossValue
0,1278_INV25002814-4,2025-07-16,1278,INV25002814-4,2.562765e+05
1,1278_25002814-4,2025-07-15,1278,25002814-4,2.562765e+05
2,1278_25002814-3,2025-07-11,1278,25002814-3,1.382700e+02
3,1278_FFHHHH2,2025-06-23,1278,FFHHHH2,2.230564e+04
4,1004_ 085219,2025-06-20,1004,085219,1.408820e+04
5,1728_25002815-2D,2025-07-11,1728,25002815-2D,1.037272e+04
6,1728_25002815-2,2025-07-11,1728,25002815-2,1.037270e+03
7,221721_111A,2025-07-17,221721,111A,5.000000e+03
8,221721_10000,2025-07-17,221721,10000,1.000000e+04
9,221721_1213,2025-07-17,221721,1213,1.000000e+04


In [28]:
vendor_ids=list(set(data['vendorId']))

In [29]:
vendor_ids

[1728, 613729, 1004, 710732, 4718, 221721, 1278]

In [30]:
supplier_data= fetch.fetch_customer_data(vendor_ids)

Sucessfully fetched data
Index(['SUPPLIER ID', 'SUPPLIER NAME', 'SUPPLIER TYPE', 'PAY TERMS', 'PHONE',
       'EMAIL_ADDRESS'],
      dtype='object')
Index(['vendorId', 'supplierName', 'supplierType', 'creditTerms', 'phone',
       'email'],
      dtype='object')


In [31]:
supplier_data

,vendorId,supplierName,supplierType,creditTerms,phone,email
0,1004,Coast Industrial & Safety Supplies Ltd,DOMESTIC SUPPLIER,30 Days Net,"722791485,,722519308",cissl@africaonline.co.ke
1,1278,Blowplast Ltd,DOMESTIC SUPPLIER,30 Days Net,"002-558649,,",sales@blowplastkenya.com
2,1728,Mega Pack (K) Ltd,DOMESTIC SUPPLIER,30 Days Net,"051-2216400,512217056,",salescor.megapack@megaspingroup.com
4,4718,Kenya Revenue Authority,TAX AUTHORITY,Immediate,None,None
5,221721,Briquette Industry Ltd,DOMESTIC SUPPLIER,30 Days Net,None,None
6,613729,Earth Kwanza Limited,DOMESTIC SUPPLIER,Immediate,None,earthkwanzaltd@gmail.com
7,710732,Sundry Creditor,DOMESTIC SUPPLIER,None,None,None


In [ ]:
      
class DataFetcherOracleSupplier:

   

    def fetch_invoice_data(self,date):

        data=pd.read_excel('storage/invoice supplier.xls')
        data['invoiceUniqueKey']=data['VENDOR_ID'].astype(str)+"_"+data[' Invoice Number']
        final_data=data[['invoiceUniqueKey','Invoice Date','VENDOR_ID',' Invoice Number','Inclusive KSH']]
        final_data=final_data[final_data['Invoice Date']>date]
        final_data=final_data[final_data['Invoice Date']<'12-02-2024']

        
        
        cleaned=final_data.rename(columns={
                                            ' Invoice Number': 'invoiceNumber',
                                            'Invoice Date': 'invoiceDate',
                                            'VENDOR_ID': 'vendorId',
                                            'Inclusive KSH':'invoiceGrossValue'
                                        })
        return cleaned.drop_duplicates()

        


        


    def fetch_supplier_data(self,vendor_ids:list):
        data=pd.read_excel('storage/data2.xls')
        
        data=data[['SUPPLIER ID','SUPPLIER NAME','SUPPLIER TYPE','PAY TERMS', 'PHONE', 'EMAIL_ADDRESS']]
        print(data.columns)
        data = data.rename(columns={
                                        'SUPPLIER ID': 'vendorId',
                                        'SUPPLIER NAME': 'supplierName',
                                        'EMAIL_ADDRESS': 'email',
                                        'PHONE': 'phone',
                                        'SUPPLIER TYPE': 'supplierType',
                                        'PAY TERMS': 'creditTerms'
                                    })
        cleaned=data[data['vendorId'].isin(vendor_ids)]
        print(cleaned.columns)
        return cleaned.drop_duplicates()

      

In [4]:
from logic import SupplierDataManger
input_data={
    14: (7,.7),
    15 :(7,.7),
    20 : (10,1),
    30 : (15,1.50),
    45 : (25,2.5),
    60 : (30,3),
    90 : (40,4), 
    365 : (65,6.5)
}

engine = create_engine("sqlite:///storage/invoice_tracker_null.db")
fetcher=SupplierDataManger(engine)
data=fetcher.offer_processor(input_data)


connection established
Sucessfully fetched data
Index(['SUPPLIER ID', 'SUPPLIER NAME', 'SUPPLIER TYPE', 'PAY TERMS', 'PHONE',
       'EMAIL_ADDRESS'],
      dtype='object')
Index(['vendorId', 'supplierName', 'supplierType', 'creditTerms', 'phone',
       'email'],
      dtype='object')
1004
1278
1728
4718
221721
613729
710732
   creditTerms  invoiceGrossValue
0           30          256276.48
1           30          256276.48
2           30             138.27
3           30           22305.64
4           30           14088.20


In [5]:
data

,invoiceUniqueKey,invoiceDate,vendorId,invoiceNumber,invoiceGrossValue,supplierId,name,email,phone,supplierType,creditTerms,createAt,modifiedAt,createdBy,modifiedBy,orginalPaymentDate,discountRate,offeredPaymentDate
0,1278_INV25002814-4,2025-07-16,1278,INV25002814-4,256276.48,2,Blowplast Ltd,sales@blowplastkenya.com,"002-558649,,",DOMESTIC SUPPLIER,30,None,None,None,None,2025-08-15,3844.14720,2025-07-31
1,1278_25002814-4,2025-07-15,1278,25002814-4,256276.48,2,Blowplast Ltd,sales@blowplastkenya.com,"002-558649,,",DOMESTIC SUPPLIER,30,None,None,None,None,2025-08-14,3844.14720,2025-07-30
2,1278_25002814-3,2025-07-11,1278,25002814-3,138.27,2,Blowplast Ltd,sales@blowplastkenya.com,"002-558649,,",DOMESTIC SUPPLIER,30,None,None,None,None,2025-08-10,2.07405,2025-07-26
3,1278_FFHHHH2,2025-06-23,1278,FFHHHH2,22305.64,2,Blowplast Ltd,sales@blowplastkenya.com,"002-558649,,",DOMESTIC SUPPLIER,30,None,None,None,None,2025-07-23,334.58460,2025-07-08
4,1004_ 085219,2025-06-20,1004,085219,14088.20,1,Coast Industrial & Safety Supplies Ltd,cissl@africaonline.co.ke,"722791485,,722519308",DOMESTIC SUPPLIER,30,None,None,None,None,2025-07-20,211.32300,2025-07-05
5,1728_25002815-2D,2025-07-11,1728,25002815-2D,10372.72,3,Mega Pack (K) Ltd,salescor.megapack@megaspingroup.com,"051-2216400,512217056,",DOMESTIC SUPPLIER,30,None,None,None,None,2025-08-10,155.59080,2025-07-26
6,1728_25002815-2,2025-07-11,1728,25002815-2,1037.27,3,Mega Pack (K) Ltd,salescor.megapack@megaspingroup.com,"051-2216400,512217056,",DOMESTIC SUPPLIER,30,None,None,None,None,2025-08-10,15.55905,2025-07-26
7,221721_111A,2025-07-17,221721,111A,5000.00,5,Briquette Industry Ltd,None,None,DOMESTIC SUPPLIER,30,None,None,None,None,2025-08-16,75.00000,2025-08-01
8,221721_10000,2025-07-17,221721,10000,10000.00,5,Briquette Industry Ltd,None,None,DOMESTIC SUPPLIER,30,None,None,None,None,2025-08-16,150.00000,2025-08-01
9,221721_1213,2025-07-17,221721,1213,10000.00,5,Briquette Industry Ltd,None,None,DOMESTIC SUPPLIER,30,None,None,None,None,2025-08-16,150.00000,2025-08-01


In [3]:
data

,invoiceUniqueKey,invoiceDate,vendorId,invoiceNumber,invoiceGrossValue,supplierId,name,email,phone,supplierType,creditTerms,createAt,modifiedAt,createdBy,modifiedBy,orginalPaymentDate,discountRate,offeredPaymentDate
0,1042_ 706 2306216723,2024-09-14,1042,706 2306216723,22675.000000,2.0,Kenya Airways,winnie.mucai@kenya-airways.com,"2125203,,",DOMESTIC SUPPLIER,30,None,None,None,None,2024-10-14,340.125000,2024-09-29
1,1042_ 706 2306216722,2024-09-14,1042,706 2306216722,22675.000000,2.0,Kenya Airways,winnie.mucai@kenya-airways.com,"2125203,,",DOMESTIC SUPPLIER,30,None,None,None,None,2024-10-14,340.125000,2024-09-29
2,1042_706 2306427842,2024-11-22,1042,706 2306427842,19425.000000,2.0,Kenya Airways,winnie.mucai@kenya-airways.com,"2125203,,",DOMESTIC SUPPLIER,30,None,None,None,None,2024-12-22,291.375000,2024-12-07
3,1042_706 2306488583,2024-11-30,1042,706 2306488583,32425.000000,2.0,Kenya Airways,winnie.mucai@kenya-airways.com,"2125203,,",DOMESTIC SUPPLIER,30,None,None,None,None,2024-12-30,486.375000,2024-12-15
4,1042_706 2306427286,2024-11-11,1042,706 2306427286,4820.000000,2.0,Kenya Airways,winnie.mucai@kenya-airways.com,"2125203,,",DOMESTIC SUPPLIER,30,None,None,None,None,2024-12-11,72.300000,2024-11-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,844734_1631,2024-12-01,844734,1631,41141.350000,16.0,JT Group Security Solutions Kenya Limited,info@jtgroupsecuritykenya.co.ke,0731000040,DOMESTIC SUPPLIER,30,None,None,None,None,2024-12-31,617.120250,2024-12-16
99,1078735_ 1164061802,2024-11-20,1078735,1164061802,988448.370082,18.0,Danisco South Africa PTY,wiberforce.thiribi@iff.com,0727038288,IMPORT SUPPLIER,30,None,None,None,None,2024-12-20,14826.725551,2024-12-05
100,1088735_VISKINV2024754,2024-09-05,1088735,VISKINV2024754,11774.000000,19.0,Vision Scientific & Engineering Kenya Limited,info@visionscientificafrica.com,+254706692768,DOMESTIC SUPPLIER,30,None,None,None,None,2024-10-05,176.610000,2024-09-20
101,1103737_0406244546,2024-06-04,1103737,0406244546,709146.000000,20.0,ABSA LIFE ASSURANCE KENYA LIMITED,ALAKCorporate@absa.africa,+254758713235,DOMESTIC SUPPLIER,30,None,None,None,None,2024-07-04,10637.190000,2024-06-19


In [2]:

from sqlmodel import SQLModel, Field, Relationship, create_engine
from logic import CustomerDataManager

engine = create_engine("sqlite:///storage/invoice_tracker_null.db")
input_data = {
    "minimum_threshold": 900000,
    "persentage_of_discount_savings": 40,
    "max_discount_amount": 9,
    "cost_of_finance_per_day": 0.2,
    14: 7,
    21: 11,
    30: 15,
    35:16,
    45: 21,
    60: 30,
    "exchange_rate": 160
    
}

offer=CustomerDataManager(engine)
data=offer.offer_processor(input_data)

connection established


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\delin\Documents\codes\pwani_finance\data_ingestion\fetch_oracle_customer.py", line 70, in fetch_invoice_data
    df = self.cursor.execute(query)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\delin\Documents\codes\pwani_finance\financePwani\Lib\site-packages\oracledb\cursor.py", line 708, in execute
    impl.execute(self)
  File "src/oracledb/impl/thick/cursor.pyx", line 318, in oracledb.thick_impl.ThickCursorImpl.execute
  File "src/oracledb/impl/thick/utils.pyx", line 484, in oracledb.thick_impl._raise_from_odpi
  File "src/oracledb/impl/thick/utils.pyx", line 474, in oracledb.thick_impl._raise_from_info
oracledb.exceptions.DatabaseError: DPY-4011: the database or network closed the connection
DPI-1080: connection was closed by ORA-03113
ORA-03113: end-of-file on communication channel
Process ID: 602533
Session ID: 37 Serial number: 31629
Help: https://docs.oracle.com/error-help/db/ora-03113/

During hand

RuntimeError: ❌ Failed to fetch invoice data: DPY-4011: the database or network closed the connection
DPI-1080: connection was closed by ORA-03113
ORA-03113: end-of-file on communication channel
Process ID: 602533
Session ID: 37 Serial number: 31629
Help: https://docs.oracle.com/error-help/db/ora-03113/

In [11]:
data

'already inserted'

In [10]:
check=data['invoiceNumber'].value_counts()

TypeError: string indices must be integers, not 'str'

In [ ]:
import pandas as pd
import oracledb
from config import oracle_config

dsn, username, password=oracle_config()
oracledb.init_oracle_client(lib_dir=r"C:\oracle\instantclient_23_8")
class OracleFetcher:
    def __init__(self):
        self.connection = None
        self.cursor = None
        self.query = None

    def connect(self):
        try:
            self.connection = oracledb.connect(
                user=username,
                password=password,
                dsn=dsn
            )
            self.cursor = self.connection.cursor()
            print("✅ Connected to Oracle DB")
        except Exception as e:
            print(f"❌ Connection failed: {e}")


            


            

    def fetch_customer_master(self,customer_number):
        try:
            
            query=f'''SELECT  DISTINCT ac.CUSTOMER_CLASS_CODE "Customer Class",
                    AC.status,
        to_char(HP.CREATION_DATE, 'DD-MM-YYYY') "Creation Date",
        HP.PARTY_TYPE "Customer Type",
        DECODE(hca.customer_type,
              'R', 'External',
              'I', 'Internal',
              hca.customer_type)            "Account Type",
        ac.CUSTOMER_NAME "Customer Name", 
        ac.CUSTOMER_NUMBER "Customer No", 
        hps.PARTY_SITE_NAME " Site Name", 
        hps.PARTY_SITE_NUMBER " Site Number", to_char(hcsu.site_use_id) site_use_id,hcas.CUST_ACCOUNT_ID,
        hcsu.STATUS "Status_S",
        HCSU.LOCATION "Location",
        substrb(look.meaning, 1, 8) "Site Use",
        HP.COUNTRY "Country",
        HP.ADDRESS1 "Address1",
        HP.ADDRESS2 "Address2",
        HP.ADDRESS3 "Address3",
        HP.ADDRESS4 "Address4",hp.PARTY_NUMBER,
        --HP.CITY "City",
        hl.CITY,hl.LOCATION_ID,
        HP.POSTAL_CODE "Postal Code",
        HP.STATE "State",
        --HP.PROVINCE "Province",
        hl.PROVINCE "Territory",
        HCA.attribute10 "Sub-Dist. Credit %",
        HCA.attribute3 "Self Collect Rebate Amount",
         hps.ATTRIBUTE1 "Transport Rebate",
         hcsu.ATTRIBUTE1 "Team Leader",
         hcsu.ATTRIBUTE2 "Trade Type",
        HCA.attribute2 "Sales Territory", --HCPA.CURRENCY_CODE "Currency",
        (SELECT RT.NAME FROM RA_TERMS_TL RT WHERE RT.TERM_ID = HCSU.PAYMENT_TERM_ID AND RT.LANGUAGE = 'US') "Terms",
        (SELECT RT.DESCRIPTION FROM RA_TERMS_TL RT WHERE RT.TERM_ID = HCSU.PAYMENT_TERM_ID AND RT.LANGUAGE = 'US') "Terms Description",
        (SELECT DISTINCT HCP.OVERALL_CREDIT_LIMIT FROM hz_cust_profile_amts hcp WHERE HCP.CUST_ACCOUNT_ID = AC.CUSTOMER_ID 
            AND HCP.SITE_USE_ID = HCSU.SITE_USE_ID AND hcp.site_use_id is not null AND HCP.CURRENCY_CODE ='KES' ) "KES Credit Limit",
        (SELECT DISTINCT HCP.OVERALL_CREDIT_LIMIT FROM hz_cust_profile_amts hcp WHERE HCP.CUST_ACCOUNT_ID = AC.CUSTOMER_ID 
            AND HCP.SITE_USE_ID = HCSU.SITE_USE_ID AND hcp.site_use_id is not null AND HCP.CURRENCY_CODE ='USD' ) "USD Credit Limit",
        (SELECT DISTINCT HCP.OVERALL_CREDIT_LIMIT FROM hz_cust_profile_amts hcp WHERE HCP.CUST_ACCOUNT_ID = AC.CUSTOMER_ID 
            AND HCP.SITE_USE_ID = HCSU.SITE_USE_ID AND hcp.site_use_id is not null AND HCP.CURRENCY_CODE = 'GBP' ) "GBP Credit Limit",
        PLH.PRICE_LIST "Pricelist Name",  hcsu.PRICE_LIST_ID, OET.TRANSACTION_TYPE_ID,hcsu.ORDER_TYPE_ID,
        OET.NAME "Order Type",
        DECODE(hcp.credit_checking, 
              'Y', 'Yes', 
              'N',  'No',
              hcp.credit_checking)   "Credit Check",hcp.CREDIT_HOLD,hcp.AUTOCASH_HIERARCHY_ID, hcp.AUTOCASH_HIERARCHY_ID_FOR_ADR,
        HCSU.FREIGHT_TERM "Freight Terms",
        (select col.Name from ar_collectors col
        where col.collector_id = hcp.collector_id) "Collector",
        srid.NAME "Default Sales Person",
 (select rid.RESOURCE_NAME from 
         JTF_RS_DEFRESOURCES_V rid where HCSU.primary_salesrep_id = srid.salesrep_id  
         and srid.RESOURCE_ID = rid.RESOURCE_ID) "Sales Person", --added by Godfrey 01/09/2016
        (select cpc.Name from hz_cust_profile_classes cpc 
        where hcp.profile_class_id=cpc.profile_class_id (+)) "Profile Class",
        AC.TAXPAYER_ID " TaxPayer ID",
        AC.TAX_REFERENCE "Tax Registration No",
        AC.ATTRIBUTE4 "Company Register ID",
(SELECT
        bank.party_name                   bank_name
        FROM hz_parties               bank
           , hz_relationships         rel
           , hz_parties               branch
           , hz_organization_profiles bank_prof
           , hz_organization_profiles branch_prof
           , iby_ext_bank_accounts    account
           , iby_external_payers_all  ext_payer
           , iby_pmt_instr_uses_all   acc_instr
           , hz_parties               cust
           , hz_cust_accounts         cust_acct
           , ar_collectors           col
           , hz_cust_site_uses_all    cust_uses
           , hz_party_sites           party_site
           , hz_locations             cust_loc
            WHERE 1=1
                AND bank.party_id                    = rel.object_id
                and bank.party_type                  = rel.object_type
                AND rel.object_table_name            = 'HZ_PARTIES'
                AND rel.relationship_code            = 'BRANCH_OF'
                AND rel.subject_id                   = branch.party_id
                AND rel.subject_type                 = branch.party_type
                AND rel.subject_table_name           = 'HZ_PARTIES'
                AND bank.party_id                    = bank_prof.party_id
                AND branch.party_id                  = branch_prof.party_id
                AND cust_acct.cust_account_id        = HCAS.cust_account_id
                AND HCAS.cust_acct_site_id      = cust_uses.cust_acct_site_id
                AND party_site.party_id              = cust.party_id
                AND party_site.party_site_id         = HCAS.party_site_id
                AND party_site.location_id           = cust_loc.location_id
                AND cust.party_id                    = cust_acct.party_id
                AND bank.party_id                    = account.bank_id
                AND branch.party_id                  = account.branch_id
                AND account.ext_bank_account_id      = acc_instr.instrument_id
                AND acc_instr.ext_pmt_party_id       = ext_payer.ext_payer_id
                AND ext_payer.cust_account_id        = cust_acct.cust_account_id
                AND cust_uses.site_use_id            = ext_payer.acct_site_use_id
                AND ROWNUM=1) "Bank Name",
(SELECT
        account.bank_account_num
        FROM hz_parties               bank
           , hz_relationships         rel
           , hz_parties               branch
           , hz_organization_profiles bank_prof
           , hz_organization_profiles branch_prof
           , iby_ext_bank_accounts    account
           , iby_external_payers_all  ext_payer
           , iby_pmt_instr_uses_all   acc_instr
           , hz_parties               cust
           , hz_cust_accounts         cust_acct
           , hz_cust_site_uses_all    cust_uses
           , hz_party_sites           party_site
           , hz_locations             cust_loc
WHERE 1=1
            AND bank.party_id                    = rel.object_id
            and bank.party_type                  = rel.object_type
            AND rel.object_table_name            = 'HZ_PARTIES'
            AND rel.relationship_code            = 'BRANCH_OF'
            AND rel.subject_id                   = branch.party_id
            AND rel.subject_type                 = branch.party_type
            AND rel.subject_table_name           = 'HZ_PARTIES'
            AND bank.party_id                    = bank_prof.party_id
            AND branch.party_id                  = branch_prof.party_id
            AND cust_acct.cust_account_id        = HCAS.cust_account_id
            AND HCAS.cust_acct_site_id      = cust_uses.cust_acct_site_id
            AND party_site.party_id              = cust.party_id
            AND party_site.party_site_id         = HCAS.party_site_id
            AND party_site.location_id           = cust_loc.location_id
            AND cust.party_id                    = cust_acct.party_id
            AND bank.party_id                    = account.bank_id
            AND branch.party_id                  = account.branch_id
            AND account.ext_bank_account_id      = acc_instr.instrument_id
            AND acc_instr.ext_pmt_party_id       = ext_payer.ext_payer_id
            AND ext_payer.cust_account_id        = cust_acct.cust_account_id
            AND cust_uses.site_use_id            = ext_payer.acct_site_use_id
            AND ROWNUM=1) "Bank A/C Number",
(SELECT
             branch.party_name                 branch_name
        FROM hz_parties               bank
           , hz_relationships         rel
           , hz_parties               branch
           , hz_organization_profiles bank_prof
           , hz_organization_profiles branch_prof
           , iby_ext_bank_accounts    account
           , iby_external_payers_all  ext_payer
           , iby_pmt_instr_uses_all   acc_instr
           , hz_parties               cust
           , hz_cust_accounts         cust_acct
           , hz_cust_site_uses_all    cust_uses
           , hz_party_sites           party_site
           , hz_locations             cust_loc
WHERE 1=1
            AND bank.party_id                    = rel.object_id
            and bank.party_type                  = rel.object_type
            AND rel.object_table_name            = 'HZ_PARTIES'
            AND rel.relationship_code            = 'BRANCH_OF'
            AND rel.subject_id                   = branch.party_id
            AND rel.subject_type                 = branch.party_type
            AND rel.subject_table_name           = 'HZ_PARTIES'
            AND bank.party_id                    = bank_prof.party_id
            AND branch.party_id                  = branch_prof.party_id
            AND cust_acct.cust_account_id        = HCAS.cust_account_id
            AND HCAS.cust_acct_site_id      = cust_uses.cust_acct_site_id
            AND party_site.party_id              = cust.party_id
            AND party_site.party_site_id         = HCAS.party_site_id
            AND party_site.location_id           = cust_loc.location_id
            AND cust.party_id                    = cust_acct.party_id
            AND bank.party_id                    = account.bank_id
            AND branch.party_id                  = account.branch_id
            AND account.ext_bank_account_id      = acc_instr.instrument_id
            AND acc_instr.ext_pmt_party_id       = ext_payer.ext_payer_id
            AND ext_payer.cust_account_id        = cust_acct.cust_account_id
            AND cust_uses.site_use_id            = ext_payer.acct_site_use_id
            AND ROWNUM=1) "Bank Branch Name",
(SELECT 
            ACV.FIRST_NAME||' '||ACV.LAST_NAME "Contact Person"
            FROM  
            AR_CONTACTS_V ACV,
            hz_parties h_contact ,
            hz_contact_points hcp,
            HZ_PARTIES HP,
            HZ_PARTIES HPP,
            HZ_CONTACT_POINTS HZP,
            HZ_ORG_CONTACTS HOC,
            HZ_RELATIONSHIPS HR,
            HZ_CUST_ACCOUNT_ROLES HCAR,
            HZ_CUST_ACCOUNTS HCA,
            HZ_PARTY_SITES HPS
        WHERE
            ACV.CONTACT_POINT_ID=HZP.CONTACT_POINT_ID
            and hr.subject_id = h_contact.PARTY_ID
            and hr.object_id = hp.party_id
            and hcp.owner_table_id(+) = hr.party_id
            and hca.party_id = hp.party_id
            and hcp.STATUS = 'A'
            AND HCA.CUST_ACCOUNT_ID = HCAS.CUST_ACCOUNT_ID 
            AND HCAS.CUST_ACCOUNT_ID = HCAR.CUST_ACCOUNT_ID
            AND HCAS.CUST_ACCT_SITE_ID = HCAR.CUST_ACCT_SITE_ID
            AND HP.PARTY_ID = HCA.PARTY_ID
            AND HPS.PARTY_ID = HCA.PARTY_ID
            AND HR.OBJECT_ID = HP.PARTY_ID
            AND HR.SUBJECT_ID = HPP.PARTY_ID
            AND HPS.PARTY_SITE_ID = HCAS.PARTY_SITE_ID
            AND HR.RELATIONSHIP_ID = HOC.PARTY_RELATIONSHIP_ID
            AND HZP.OWNER_TABLE_ID = HCAR.PARTY_ID
            AND HOC.ORG_CONTACT_ID=ACV.ORG_CONTACT_ID
            AND ROWNUM=1) "Contact Person",
(
SELECT 
            ACV.JOB_TITLE "Job Title" 
        FROM  
            AR_CONTACTS_V ACV,
            hz_parties h_contact ,
            hz_contact_points hcp,
            HZ_PARTIES HP,
            HZ_PARTIES HPP,
            HZ_CONTACT_POINTS HZP,
            HZ_ORG_CONTACTS HOC,
            HZ_RELATIONSHIPS HR,
            HZ_CUST_ACCOUNT_ROLES HCAR,
            HZ_CUST_ACCOUNTS HCA,
            HZ_PARTY_SITES HPS
    WHERE
            ACV.CONTACT_POINT_ID=HZP.CONTACT_POINT_ID
            and hr.subject_id = h_contact.PARTY_ID
            and hr.object_id = hp.party_id
            and hcp.owner_table_id(+) = hr.party_id
            and hca.party_id = hp.party_id
            and hcp.STATUS = 'A'
            AND HCA.CUST_ACCOUNT_ID = HCAS.CUST_ACCOUNT_ID 
            AND HCAS.CUST_ACCOUNT_ID = HCAR.CUST_ACCOUNT_ID
            AND HCAS.CUST_ACCT_SITE_ID = HCAR.CUST_ACCT_SITE_ID
            AND HP.PARTY_ID = HCA.PARTY_ID
            AND HPS.PARTY_ID = HCA.PARTY_ID
            AND HR.OBJECT_ID = HP.PARTY_ID
            AND HR.SUBJECT_ID = HPP.PARTY_ID
            AND HPS.PARTY_SITE_ID = HCAS.PARTY_SITE_ID
            AND HR.RELATIONSHIP_ID = HOC.PARTY_RELATIONSHIP_ID
            AND HZP.OWNER_TABLE_ID = HCAR.PARTY_ID
            AND HOC.ORG_CONTACT_ID=ACV.ORG_CONTACT_ID
            AND ROWNUM=1) "Job Title",
(SELECT
        DISTINCT PHONE_COUNTRY_CODE||'-'||PHONE_AREA_CODE||'-'||PHONE_NUMBER
        FROM 
            HZ_PARTIES HP,
            HZ_PARTIES HPP,
            HZ_CONTACT_POINTS HZP,
            HZ_ORG_CONTACTS HOC,
            HZ_RELATIONSHIPS HR,
            HZ_CUST_ACCOUNT_ROLES HCAR,
            HZ_CUST_ACCOUNTS HCA,
            HZ_PARTY_SITES HPS
    WHERE 
            HCA.CUST_ACCOUNT_ID = HCAS.CUST_ACCOUNT_ID 
        AND HCAS.CUST_ACCOUNT_ID = HCAR.CUST_ACCOUNT_ID
        AND HCAS.CUST_ACCT_SITE_ID = HCAR.CUST_ACCT_SITE_ID
        AND HP.PARTY_ID = HCA.PARTY_ID
        AND HPS.PARTY_ID = HCA.PARTY_ID
        AND HR.OBJECT_ID = HP.PARTY_ID
        AND HR.SUBJECT_ID = HPP.PARTY_ID
        AND HPS.PARTY_SITE_ID = HCAS.PARTY_SITE_ID
        AND HR.RELATIONSHIP_ID = HOC.PARTY_RELATIONSHIP_ID
        AND HZP.OWNER_TABLE_ID = HCAR.PARTY_ID
        AND HP.PARTY_ID = HPSEL.PARTY_ID
        AND HP.PARTY_TYPE='ORGANIZATION'
        AND HZP.CONTACT_POINT_TYPE = 'PHONE'
        AND ROWNUM = 1) "Phone Number",
(SELECT
        DISTINCT HZP.EMAIL_ADDRESS
    FROM 
        HZ_PARTIES HP,
        HZ_PARTIES HPP,
        HZ_CONTACT_POINTS HZP,
        HZ_ORG_CONTACTS HOC,
        HZ_RELATIONSHIPS HR,
        HZ_CUST_ACCOUNT_ROLES HCAR,
        HZ_CUST_ACCOUNTS HCA,
        HZ_PARTY_SITES HPS
WHERE 
            HCA.CUST_ACCOUNT_ID = HCAS.CUST_ACCOUNT_ID 
        AND HCAS.CUST_ACCOUNT_ID = HCAR.CUST_ACCOUNT_ID
        AND HCAS.CUST_ACCT_SITE_ID = HCAR.CUST_ACCT_SITE_ID
        AND HP.PARTY_ID = HCA.PARTY_ID
        AND HPS.PARTY_ID = HCA.PARTY_ID
        AND HR.OBJECT_ID = HP.PARTY_ID
        AND HR.SUBJECT_ID = HPP.PARTY_ID
        AND HPS.PARTY_SITE_ID = HCAS.PARTY_SITE_ID
        AND HR.RELATIONSHIP_ID = HOC.PARTY_RELATIONSHIP_ID
        AND HZP.OWNER_TABLE_ID = HCAR.PARTY_ID
        AND HP.PARTY_TYPE='ORGANIZATION'
        AND HZP.CONTACT_POINT_TYPE = 'EMAIL'
        AND HP.PARTY_ID = HPSEL.PARTY_ID
        AND ROWNUM=1
        ) "Email Address",
       hp.PARTY_ID,hp.EMAIL_ADDRESS 
--=============================================================================
--Tables
-------------------------------------------------------------------------------
FROM 
        AR_CUSTOMERS AC,
        ra_salesreps_all        srid,
        HZ_CUST_ACCT_SITES_ALL HCAS, 
        hz_cust_accounts HCA, 
        HZ_CUST_SITE_USES_ALL HCSU, 
        HZ_PARTY_SITES HPS,
        hz_parties HP,
        HZ_LOCATIONS hl,
        hz_parties HPSEL,
        ar_lookups look,
        ar_lookups look_status,
        OE_Price_Lists_Active_V PLH,
        OE_TRANSACTION_TYPES_TL OET, --hz_cust_profile_amts hcpa,
        hz_customer_profiles    hcp
        
--=============================================================================
WHERE   AC.CUSTOMER_ID = HCAS.CUST_ACCOUNT_ID
        AND HCAS.CUST_ACCT_SITE_ID = HCSU.CUST_ACCT_SITE_ID
        AND HCAS.PARTY_SITE_ID = HPS.PARTY_SITE_ID
        AND HCA.CUST_ACCOUNT_ID = AC.CUSTOMER_ID
        AND hca.cust_account_id    =  hcp.cust_account_id
        AND HCA.PARTY_ID = HP.PARTY_ID
        AND look.lookup_type(+) = 'SITE_USE_CODE'
        AND look.lookup_code(+) = HCSU.SITE_USE_CODE
        AND look_status.lookup_type(+) = 'CODE_STATUS'
        AND look_status.lookup_code(+) = nvl(AC.status, 'A')
        AND HCSU.PRICE_LIST_ID =PLH.PRICE_LIST_ID(+)
        AND HCSU.primary_salesrep_id         = srid.salesrep_id(+)
        and HCSU.ORDER_TYPE_ID = OET.TRANSACTION_TYPE_ID (+)
        AND HP.PARTY_ID = HPSEL.PARTY_ID 
        AND hps.LOCATION_ID = hl.LOCATION_ID
        --and   HCPA.CUST_ACCOUNT_ID = AC.CUSTOMER_ID
        --AND HCPA.CURRENCY_CODE is not null 
--        AND HCPA.SITE_USE_ID --= HCSU.SITE_USE_ID 
--        in ('8601',
--'8600',
--'453574',
--'453574',
--'1242',
--'1241',
--'1243',
--'1240',
--'15065',
--'597591',
--'597589',
--'517583',
--'364567',
--'1247',
--'623589',
--'580578',
--'580578',
--'520571',
--'453568',
--'453568',
--'274557',
--'546573',
--'5819',
--'4034',
--'8621',
--'8620',
--'453571',
--'6215',
--'13921',
--'1276',
--'12260',
--'8384',
--'1280',
--'1622',
--'1621',
--'1622',
--'1621',
--'19355',
--'19352',
--'425576',
--'6414',
--'9200',
--'9200',
--'287553',
--'287555',
--'1184',
--'8485',
--'5922',
--'246549') 
       --AND HP.CREATION_DATE BETWEEN NVL (:p_from_date, HP.CREATION_DATE)
                                --AND NVL (:p_to_date, HP.CREATION_DATE)
        --AND AC.CUSTOMER_NAME=:C_Name
        --AND HPS.PARTY_SITE_NAME=:SITE_NAME
        and AC.status = 'A'
        and hcsu.STATUS = 'A'
        AND AC.CUSTOMER_NUMBER IN {customer_number}'''
            # df=pd.read_sql_query(query,self.connection)
            # return df 
            df = self.cursor.execute(query)
            rows = df.fetchall()
            columns = [desc[0] for desc in self.cursor.description]  # Get column names

            import pandas as pd
            df = pd.DataFrame(rows, columns=columns)
            return df

        except Exception as e:
            print(f"❌ Query failed: {e}")
        # finally:
        #     if self.cursor:
        #         self.cursor.close()
        #     if self.connection:
        #         self.connection.close()


    def fetch_invoice_data(self,date):
        try:
            
            query=f"""SELECT  --NAME, CUSTOMER_NAME, ORDER_NUMBER,INVOICE_NUMBER, PERIOD  , INVOICE_DATE,  ,INVOICE_AMOUNT
PB.* 
,(select ROUND(costvu.item_cost,5) from CST_ITEM_COST_TYPE_V costvu
where costvu.INVENTORY_ITEM_ID = pb.INVENTORY_ITEM_ID
AND pb.ORGANIZATION_ID = COSTVU.ORGANIZATION_ID
AND costvu.item_cost<>0 and costvu.COST_TYPE_ID = 2)UNIT_COST
-------------------------------------------
,(SELECT distinct MDEV.ELEMENT_VALUE FROM MTL_DESCR_ELEMENT_VALUES_V MDEV WHERE MDEV.INVENTORY_ITEM_ID=PB.INVENTORY_ITEM_ID AND MDEV.ELEMENT_NAME='Product Code' and rownum=1) "Product Code"
,(SELECT distinct  MDEV.ELEMENT_VALUE FROM MTL_DESCR_ELEMENT_VALUES_V MDEV WHERE MDEV.INVENTORY_ITEM_ID=PB.INVENTORY_ITEM_ID AND MDEV.ELEMENT_NAME='Product Pack'  and rownum=1) "Product Pack"
,
--------
(SELECT distinct MDEV.ELEMENT_VALUE FROM MTL_DESCR_ELEMENT_VALUES_V MDEV WHERE MDEV.INVENTORY_ITEM_ID=PB.INVENTORY_ITEM_ID AND MDEV.ELEMENT_NAME='Size' and rownum=1)   "SKU Size",
(SELECT distinct MDEV.ELEMENT_VALUE FROM MTL_DESCR_ELEMENT_VALUES_V MDEV WHERE MDEV.INVENTORY_ITEM_ID=PB.INVENTORY_ITEM_ID AND MDEV.ELEMENT_NAME='Additional Description' and rownum=1) "Variant",
(SELECT distinct MDEV.ELEMENT_VALUE FROM MTL_DESCR_ELEMENT_VALUES_V MDEV WHERE MDEV.INVENTORY_ITEM_ID=PB.INVENTORY_ITEM_ID AND MDEV.ELEMENT_NAME='Sub Variant' and rownum=1) "Sub Variant"
--,SOLD_TO_CUSTOMER_ID,rcta.SOLD_TO_CUSTOMER_ID,
,ac.CUSTOMER_NUMBER "Customer No"
FROM APPS.XX_PWANIBRAND_VIEW PB,ra_customer_trx_lines_all RCTL ,ra_customer_trx_all rcta,AR_CUSTOMERS AC
WHERE 
TRUNC(PB.INVOICE_DATE) >= TO_DATE({date}, 'DD-MM-YYYY')
and PB.customer_trx_id = rctl.customer_trx_id(+)
and PB.CUSTOMER_TRX_LINE_ID = rctl.LINK_TO_CUST_TRX_LINE_ID (+)
and PB.customer_trx_id = rcta.CUSTOMER_TRX_ID (+)
and rcta.SOLD_TO_CUSTOMER_ID = ac.CUSTOMER_ID (+)"""
            df = self.cursor.execute(query)
            rows = df.fetchall()
            columns = [desc[0] for desc in self.cursor.description]  # Get column names

            import pandas as pd
            df = pd.DataFrame(rows, columns=columns)
            return df
            # df=pd.read_sql_query(query,self.connection)
            # return df 

        except Exception as e:
            print(f"❌ Query failed: {e}")
        # finally:
        #     if self.cursor:
        #         self.cursor.close()
        #     if self.connection:
        #         self.connection.close()  

    def pipline():
        pass                      

In [ ]:
def fetch_invoice_data(date):

        data=pd.read_excel('storage/Supplier Invoice data.xls')
        final_data=data[['INVOICE_NUMBER','INVOICE_DATE','Customer No','INVOICE_GROSS_VALUE','INVOICE_CURRENCY_CODE']]
        final_data=final_data[final_data['INVOICE_DATE']==date]
        
        cleaned=final_data.groupby(['INVOICE_NUMBER']).agg({
                                                        'INVOICE_DATE': 'first',
                                                        'Customer No': 'first',
                                                        'INVOICE_GROSS_VALUE': 'sum',
                                                        'INVOICE_CURRENCY_CODE': 'first'
                                                        }).reset_index()
        cleaned=cleaned.rename(columns={
                                            'INVOICE_NUMBER': 'invoiceNumber',
                                            'INVOICE_DATE': 'invoiceDate',
                                            'Customer No': 'customerNumber',
                                            'INVOICE_GROSS_VALUE': 'creditAmount',
                                            'INVOICE_CURRENCY_CODE':'invoiceCurrencyCode'
                                        })
        return cleaned

        


        


def fetch_customer_data(customer_nos:list):
        data=pd.read_excel('storage/Customer_master_data.xls')
        
        data=data[['Customer No','Customer Name','EMAIL_ADDRESS','Phone Number','Customer Type','Terms']]
        print(data.columns)
        data = data.rename(columns={
                                        'Customer No': 'customerNumber',
                                        'Customer Name': 'name',
                                        'EMAIL_ADDRESS': 'email',
                                        'Phone Number': 'phone',
                                        'Customer Type': 'customerType',
                                        'Terms': 'creditTerms'
                                    })
        cleaned=data[data['customerNumber'].isin(customer_nos)]
        cleaned=cleaned.groupby(['customerNumber']).agg({'name': 'first',
                                        'email': 'first',
                                        'phone': 'first',
                                        'customerType': 'first',
                                        'creditTerms': 'max'}).reset_index()
        print(cleaned.columns)
        return cleaned

      

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
